In [ ]:
from keras.datasets import mnist
from google.colab import files
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import LambdaCallback
from keras import backend as K
import keras
import numpy as np
from PIL import Image
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from scipy.stats import pearsonr, spearmanr

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# ratio of the dataset you wanan use
ratio = 1
subset_size = int(ratio * len(X_train))
X_train = X_train[:subset_size].reshape((subset_size, -1))/255.0
y_train = to_categorical(y_train[:subset_size], 10)

subset_size = int(ratio * len(X_test))
X_test = X_test[:subset_size].reshape((subset_size, -1))/255.0
y_test = to_categorical(y_test[:subset_size], 10)


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(128, activation='relu', input_shape=(128,)))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ds_size = 60000  * ratio
batchsize_train = 128
dense_size = 128
n_batches = int(ds_size // batchsize_train)
n_epochs = 3
n_layers = 2

In [ ]:
shapes = (n_batches, batchsize_train, dense_size)

layer_activations = []
for _ in range(n_layers):
  epochs = []
  for i in range(n_epochs):
    epochs.append(np.empty(shapes))
  layer_activations.append(epochs)
model_output = []
weights = []

class CustomCallback(Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        for i in range(n_layers):
          model_output.append(model.layers[i].output)
        self.ok = False #used for testing

    def on_epoch_begin(self, epoch, logs=None):
      self.neuron_activations = []
      for i in range(n_layers):
        self.neuron_activations.append(layer_activations[i][epoch])
      if epoch == 0: ## replace 0 with the epoch you wanna test
        self.ok = True

    def on_batch_begin(self, batch, logs=None):
        get_activations = K.function([model.layers[0].input], model_output)
        if batch != (n_batches):  # we ignore the last, uncompleted batch so we can have a matrix
          get_act = get_activations([X_train[(batch) * batchsize_train: (batch + 1) * batchsize_train]])
          for i in range(n_layers):
            activations = get_act[i]
            self.neuron_activations[i][batch]= np.array(activations)
        if batch == 50 and self.ok == True:
           weights.append(model.get_weights())
           self.ok = False

custom_callback = CustomCallback()

model.fit(X_train, y_train, epochs=n_epochs, batch_size=batchsize_train, callbacks=[custom_callback])


Epoch 1/3
  5/469 [..............................] - ETA: 12s - loss: 2.1618 - accuracy: 0.2859

469/469 [==============================] - 13s 21ms/step - loss: 0.3192 - accuracy: 0.9090
Epoch 2/3
469/469 [==============================] - 6s 12ms/step - loss: 0.1275 - accuracy: 0.9625
Epoch 3/3
469/469 [==============================] - 4s 8ms/step - loss: 0.0877 - accuracy: 0.9734


Properly arrange list

In [ ]:
neuron_activations = []
for j in range(n_layers):
  for i in range(n_epochs):
    layer_activations[j][i] = np.reshape(layer_activations[j][i], (n_batches * batchsize_train, dense_size ))
    layer_activations[j][i] = layer_activations[j][i].T.reshape((dense_size,-1))
  neuron_activations.append(np.hstack(layer_activations[j]))


Remove top and bottom 5%

In [ ]:
ammount_removed = 0.025

In [ ]:
remove_count = int(len(neuron_activations[0][0]) * ammount_removed)
neuron_activations_no_outliers = np.empty((2,128, (len(neuron_activations[0][0]) - 2 * remove_count)))
for layer_ in range(n_layers):
  for neuron_ in range(dense_size):
    sorted_indices = np.argsort(neuron_activations[layer_][neuron_])
    top_indices = sorted_indices[-remove_count:]
    bottom_indices = sorted_indices[:remove_count]
    trimmed = np.delete(neuron_activations[layer_][neuron_], np.concatenate((top_indices, bottom_indices)))
    neuron_activations_no_outliers[layer_][neuron_] = trimmed

In [ ]:
neuron_activations = neuron_activations_no_outliers

We now compute the ranges of the neurons

In [ ]:
ranges = np.empty((n_layers,dense_size,2))
for layer in range(n_layers):
  for i in range(dense_size):
    ranges[layer][i][0] = np.min(neuron_activations[layer][i])
    ranges[layer][i][1] = np.max(neuron_activations[layer][i])


Get activations from tests

In [ ]:
get_testing_a = K.function([model.layers[0].input],
                                  model_output)
get_label_pred = K.function([model.layers[0].input],
                                  [model.layers[-1].output])
layers_output = get_testing_a([X_test])
label_pred = get_label_pred([X_test])[0]

In [ ]:
max_indices = np.argmax(label_pred, axis=1) # having the right label eg 7 or 4
y_test_predict = np.zeros((max_indices.size, 10))
y_test_predict[np.arange(max_indices.size), max_indices] = 1 # one hot encoded

Rearrange the list properly

In [ ]:
layers_output= np.array(layers_output)
layers_output = layers_output.transpose(0, 2, 1)

In [ ]:
n_images = len(layers_output[0][0])
percentages = np.empty(n_images)
for i in range(n_images):
  count = 0
  for layer in range(n_layers):
    for neuron in range(dense_size):
        if ranges[layer][neuron][0] <= layers_output[layer][neuron][i] <= ranges[layer][neuron][1] :
          count +=1
  percentages[i] = count / (n_layers * dense_size) * 100

Compute Correlation and Accuracy

In [ ]:
result = np.equal(y_test, y_test_predict)

result = result.all(axis=1)


In [ ]:
uploaded = files.upload()

Saving 00.png to 00.png
Saving 0.png to 0.png
Saving 1.png to 1.png
Saving 2.png to 2.png
Saving 3.png to 3.png
Saving 4.png to 4.png
Saving 5.png to 5.png
Saving 6.png to 6.png
Saving 7.png to 7.png
Saving 8.png to 8.png
Saving 9.png to 9.png
Saving 11.png to 11.png
Saving 22.png to 22.png
Saving 33.png to 33.png
Saving 44.png to 44.png
Saving 55.png to 55.png
Saving 66.png to 66.png
Saving 77.png to 77.png
Saving 88.png to 88.png
Saving 99.png to 99.png


In [ ]:
filenames = ["0.png","00.png", "1.png", "11.png","2.png","22.png","3.png","33.png", "4.png","44.png", "5.png","55.png", "6.png","66.png", "7.png","77.png", "8.png","88.png", "9.png","99.png"]
# filenames = ["0.png", "1.png", "2.png", "3.png", "4.png", "5.png", "6.png", "7.png", "8.png", "9.png"]
# filenames = ["00.png", "11.png", "22.png", "33.png", "44.png", "55.png", "66.png", "77.png", "88.png", "99.png"]

In [ ]:
final_arrays_list = np.empty((2,784))

for i in range(len(filenames)):
    filename = filenames[i]
    image = Image.open(filename)
    image = image.resize((28, 28))
    image = image.convert("L")
    image_array = np.array(image)

    inverted_image_array = 1 - (image_array / 255.0)

    flattened_image_array = inverted_image_array.flatten()

    X_train[i] = flattened_image_array

final = np.concatenate((final_arrays_list[0], final_arrays_list[1]), axis=0)


In [ ]:
number = len(filenames)
layers_output = get_testing_a([X_train[:number]])
label_pred = get_label_pred(X_train[:number])[0]

In [ ]:
max_indices = np.argmax(label_pred, axis=1) # having the right label eg 7 or 4
y_test_predict = np.zeros((max_indices.size, 10))
y_test_predict[np.arange(max_indices.size), max_indices] = 1 # one hot encoded

In [ ]:
layers_output= np.array(layers_output)
layers_output = layers_output.transpose(0, 2, 1)

In [ ]:
n_images = len(layers_output[0][0])
percentages = np.empty(n_images)
for i in range(n_images):
  count = 0
  for layer in range(n_layers):
    for neuron in range(dense_size):
        if ranges[layer][neuron][0] <= layers_output[layer][neuron][i] <= ranges[layer][neuron][1] :
          count +=1
  percentages[i] = count / (n_layers * dense_size) * 100

In [ ]:
print(percentages)

[ 99.609375  92.578125  99.609375  93.75      99.609375  85.9375
  99.21875   81.640625  98.828125  94.140625  98.4375    93.359375
  99.21875   90.625     98.828125  85.546875 100.        90.234375
  99.609375  91.40625 ]


In [ ]:
max_indices = np.argmax(label_pred, axis=1) # having the right label eg 7 or 4
y_test_predict = np.zeros((number, 10))
y_test_predict[np.arange(max_indices.size), max_indices] = 1 # one hot encoded

In [ ]:
y_test_predict

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [ ]:
def find_corr(array_v, array_bool):
  numeric_boolean_values = array_bool.astype(int)
  pearson_corr, pearson_p_value = pearsonr(array_v, numeric_boolean_values)
  spearman_corr, spearman_p_value = spearmanr(array_v, numeric_boolean_values)
  return pearson_corr, pearson_p_value, spearman_corr, spearman_p_value

In [ ]:
results = np.empty(len(y_test_predict), dtype = bool)
for i in range(len(y_test_predict)):
  results[i] = (y_test_predict[i][i//2] == 1)

In [ ]:
print(results)
find_corr2(percentages, results)
find_corr(percentages, results)

[False False  True False  True False  True  True False False False  True
  True False  True False False False False False]


(0.1765488101957695,
 0.45651921434766546,
 0.21914564294084654,
 0.353253042063813)